In [1]:
import pandas as pd

champ_2023 = pd.read_csv(r"C:\Users\Owner\dev\football-analytics\data-scraping\fbref\scrape-game-shots\shot_data_champ_2023.csv")
champ_2024 = pd.read_csv(r"C:\Users\Owner\dev\football-analytics\data-scraping\fbref\scrape-game-shots\shot_data_champ_2024.csv")
prem_2024 = pd.read_csv(r"C:\Users\Owner\dev\football-analytics\data-scraping\fbref\scrape-game-shots\shot_data_prem_2024.csv")
prem_2023 = pd.read_csv(r"C:\Users\Owner\dev\football-analytics\data-scraping\fbref\scrape-game-shots\shot_data_prem_2023.csv")

df = pd.concat([champ_2023, prem_2024, prem_2023])

# Remove rows where Team is 0
df = df[df['Team'] != 0]
df = df[df['Team'] != '0']  # in case it's stored as string


df

,Minute,Team,Player,Event Type,Outcome,xG,PSxG,match_url,match_date,home_team,away_team,division
1,5,Sheffield Weds,Barry Bannan,Shot,Off Target,0.03,0.00,https://fbref.com/en/matches/bb5199b8/Sheffiel...,2023-08-04,Sheffield Weds,Southampton,EFL Championship
2,5,Sheffield Weds,Juan Delgado,Shot,Blocked,0.09,0.00,https://fbref.com/en/matches/bb5199b8/Sheffiel...,2023-08-04,Sheffield Weds,Southampton,EFL Championship
3,5,Sheffield Weds,Lee Gregory,Shot,Blocked,0.21,0.00,https://fbref.com/en/matches/bb5199b8/Sheffiel...,2023-08-04,Sheffield Weds,Southampton,EFL Championship
4,5,Sheffield Weds,Callum Paterson,Shot,Off Target,0.03,0.00,https://fbref.com/en/matches/bb5199b8/Sheffiel...,2023-08-04,Sheffield Weds,Southampton,EFL Championship
5,8,Southampton,Nathan Tella,Shot,Blocked,0.04,0.00,https://fbref.com/en/matches/bb5199b8/Sheffiel...,2023-08-04,Sheffield Weds,Southampton,EFL Championship
...,...,...,...,...,...,...,...,...,...,...,...,...
10896,77,Nott'ham Forest,Anthony Elanga,Shot,Saved,0.06,0.02,https://fbref.com/en/matches/dafb05fe/Burnley-...,2024-05-19,Burnley,Nott'ham Forest,Premier League
10897,78,Burnley,Vitinho,Shot,Off Target,0.06,0.00,https://fbref.com/en/matches/dafb05fe/Burnley-...,2024-05-19,Burnley,Nott'ham Forest,Premier League
10898,82,Burnley,Jóhann Berg Guðmundsson,Shot,Saved,0.05,0.05,https://fbref.com/en/matches/dafb05fe/Burnley-...,2024-05-19,Burnley,Nott'ham Forest,Premier League
10899,88,Burnley,Wilson Odobert,Shot,Off Target,0.05,0.00,https://fbref.com/en/matches/dafb05fe/Burnley-...,2024-05-19,Burnley,Nott'ham Forest,Premier League


In [6]:
import pandas as pd
import numpy as np

match_stats = []

for match_url, match_events in df.groupby('match_url'):
    # Get match metadata from first event
    first_event = match_events.iloc[0]
    home_team = first_event['home_team']
    away_team = first_event['away_team']
    match_date = first_event['match_date']
    division = first_event['division']  # Added division
    
    # Calculate goals (shots with outcome "Goal")
    home_goals = match_events[
        (match_events['Team'] == home_team) & 
        (match_events['Outcome'] == 'Goal')
    ].shape[0]
    
    away_goals = match_events[
        (match_events['Team'] == away_team) & 
        (match_events['Outcome'] == 'Goal')
    ].shape[0]
    
    # Calculate xG (sum of xG for all shots)
    home_xg = match_events[
        (match_events['Team'] == home_team) 
    ]['xG'].sum()
    
    away_xg = match_events[
        (match_events['Team'] == away_team) 
    ]['xG'].sum()
    
    # Get penalty minutes
    penalty_minutes = set(match_events[match_events['Event Type'] == 'Penalty']['Minute'])
    
    # Get red card minutes - filtering out those that coincide with penalties
    home_red_cards = match_events[
        (match_events['Team'] == home_team) & 
        (match_events['Outcome'] == 'Red Card')
    ]
    home_red_mins = home_red_cards[~home_red_cards['Minute'].isin(penalty_minutes)]['Minute'].map(lambda x: 90 - x).sum()
    
    away_red_cards = match_events[
        (match_events['Team'] == away_team) & 
        (match_events['Outcome'] == 'Red Card')
    ]
    away_red_mins = away_red_cards[~away_red_cards['Minute'].isin(penalty_minutes)]['Minute'].map(lambda x: 90 - x).sum()
    
    match_stats.append({
        'match_url': match_url,
        'match_date': match_date,
        'division': division,  # Added division to output
        'home_team': home_team,
        'away_team': away_team,
        'home_goals': home_goals,
        'away_goals': away_goals,
        'home_xg': round(home_xg, 2),
        'away_xg': round(away_xg, 2),
        'home_red_card_mins': home_red_mins,
        'away_red_card_mins': away_red_mins
    })

match_stats_df = pd.DataFrame(match_stats)

# Convert match_date to datetime if it isn't already
match_stats_df['match_date'] = pd.to_datetime(match_stats_df['match_date'])

# Create season column with correct year formatting based on match date
match_stats_df['season'] = match_stats_df['match_date'].apply(
    lambda x: int(f"{x.year-1}{x.year}") if x.month < 8 else int(f"{x.year}{x.year+1}")
)


print(match_stats_df.sort_values(by="home_red_card_mins", ascending=False).head())

                                              match_url match_date  \
761   https://fbref.com/en/matches/a436996c/Liverpoo... 2024-12-14   
16    https://fbref.com/en/matches/02e9a08c/Swansea-... 2023-11-04   
955   https://fbref.com/en/matches/ccdda6d3/Bournemo... 2025-02-22   
1059  https://fbref.com/en/matches/e4480630/Southamp... 2024-12-04   
975   https://fbref.com/en/matches/d11e033a/Sunderla... 2023-10-07   

              division     home_team      away_team  home_goals  away_goals  \
761     Premier League     Liverpool         Fulham           2           2   
16    EFL Championship  Swansea City     Sunderland           0           0   
955     Premier League   Bournemouth         Wolves           0           1   
1059    Premier League   Southampton        Chelsea           1           5   
975   EFL Championship    Sunderland  Middlesbrough           0           4   

      home_xg  away_xg  home_red_card_mins  away_red_card_mins    season  
761      2.14     1.22       

In [3]:
match_stats_df.to_csv("shot_and_red_card_data.csv", index=False)